In [1]:
from __future__ import print_function, division
import os
import time
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
torch.set_default_dtype(torch.float32)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.backends.cudnn.enabled

from network import UNet as Model
from Config import config as Config
from AugBlock import augBlock
from gtaV_helper import GTA5
from cityscapes_helper import CitySpaceseDataset

In [7]:


RESULT_FOLDER = '/Result'
TENSORBOARD_PREFIX = f'{RESULT_FOLDER}/tensorboard'

def iou(outputs, labels):
    outputs = outputs.squeeze(1) 
    
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded

def CEL(pred,target):
    criterion = nn.CrossEntropyLoss()
    loss = criterion(pred,target)
    return loss

def train(model, train_loader, aug_train_loader, optimizer, scheduler, writer, cfg=None, test_loader = None):  
    best_loss = 1e10
    best_val = 100
    count = 0

    # looping over given number of epochs
    for epoch in range(cfg.num_epochs):
        tic = time.time()
        # scheduler.step()
        model.train()

        for inputs, targets in train_loader:
            count += 1

            inputs = inputs.cuda()
            targets = targets.cuda()

            optimizer.zero_grad()
            preds = model(inputs)
            loss = CEL(preds, targets)

            loss.backward()
            optimizer.step()

            # tensorboard logging
            writer.add_scalar('Train/Loss', loss.item(), count)

            print(f'Epoch:{epoch}, Step:{count}, Loss:{loss.item():.6f}, BestVal:{best_val:.6f}, Time:{time.time()-tic}')
            tic = time.time()


        for inputs, targets in aug_train_loader:
            count += 1

            inputs = inputs.cuda()
            targets = targets.cuda()

            optimizer.zero_grad()
            preds = model(inputs)
            loss = CEL(preds, targets)

            loss.backward()
            optimizer.step()

            # tensorboard logging
            writer.add_scalar('Train/Loss', loss.item(), count)

            print(f'Epoch:{epoch}, Step:{count}, Loss:{loss.item():.6f}, BestVal:{best_val:.6f}, Time:{time.time()-tic}')
            tic = time.time()

        if cfg.val_freq != None and epoch%cfg.val_freq==0:
            val_loss, best_val = eval_model(model, test_loader, best_val=best_val, cfg=cfg)
            writer.add_scalar('Val/Loss', val_loss, count)
        
    return best_val

def eval_model(model, test_loader,best_val=100, cfg=None):

    # Set model to evaluate mode
    model.eval()

    n_samples = 0
    avg_loss = 0

    # check dataset type
    for inputs, targets in test_loader:
        inputs = inputs.cuda()
        targets = targets.cuda()

        with torch.set_grad_enabled(False):
            preds = model(inputs)  
            preds = torch.clip(preds, 0, 1)

            loss = iou(preds, targets) 
            # NMSE

            avg_loss += (loss.item() * inputs.shape[0])
            n_samples += inputs.shape[0]

    avg_loss = avg_loss / (n_samples + 1e-7)

    if avg_loss < best_val:
        best_val = avg_loss
        # save ckpt
        torch.save(model.state_dict(), f'{RESULT_FOLDER}/{cfg.exp_name}/model_{best_val:.5f}.pt')
        print(f'[*] model saved to: {RESULT_FOLDER}/{cfg.exp_name}/model_{best_val:.5f}.pt')

    model.train()
    return avg_loss, best_val


# def train_test_run(cfg,writer,train_path,test_path):
#     cfg = Config()
    
#     os.makedirs(TENSORBOARD_PREFIX, exist_ok=True)
#     os.makedirs(f'{RESULT_FOLDER}/{cfg.exp_name}', exist_ok=True)   

#     writer = SummaryWriter(log_dir=f'{TENSORBOARD_PREFIX}/{cfg.exp_name}')
    
#     data_train = Cityscapes_data(train_path, split='train', mode='fine',target_type='semantic')
#     data_test = GTA5(test_path)
        
#     train_loader =  DataLoader(data_train, batch_size=cfg.batch_size, shuffle=True, drop_last=True, num_workers=cfg.num_workers, pin_memory=False, generator=torch.Generator(device='cuda'))
#     test_loader =  DataLoader(data_test, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers, pin_memory=False, generator=torch.Generator(device='cuda'))
        

#     # init model 
#     model = Model()
#     model.cuda()

#     # init optimizer and scheduler
#     optimizer = optim.Adam(model.parameters(), lr=cfg.lr)
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=cfg.step, gamma=cfg.lr_decay)

#     best_val_train = train(model, train_loader, optimizer, scheduler, writer, cfg=cfg)

#     print('[*] train ends... ')
#     print(f'[*] best val loss train: {best_val_train}')

#     final_loss, best_val_test = (model, test_loader, best_val=100, cfg=None)
#     print('[*] Eval ends... ')
#     print(f'[*] best val loss test: {best_val_test}')
    
#     return best_val_test


In [2]:
from gtaV_helper import GTA5
import matplotlib.pyplot as plt

In [3]:
data_train = GTA5('B:/Notes/SEM 3/EE641/Project/GTA5/data/')

In [4]:
from torch.utils.data import DataLoader
train_loader =  DataLoader(data_train, batch_size=5, shuffle=True, drop_last=True, num_workers=0, pin_memory=False, generator=torch.Generator(device='cuda'))

In [5]:
inputs, targets = next(iter(train_loader))

In [6]:
from AugBlock import augBlock

In [7]:
inputs.cpu().size()

torch.Size([5, 256, 455, 3])

In [8]:
targets.cpu().size()

torch.Size([5, 256, 455])

In [9]:
inputs = np.transpose(inputs.cpu(), [0,3,1,2]) 

In [10]:
dp = augBlock(inputs.cpu().numpy(), targets.cpu().numpy())

In [25]:
aug_inputs, aug_targets = next(iter(dp))

torch.Size([5, 3, 256, 455]) torch.Size([5, 256, 455])


In [11]:
inputs2, labels = next(iter(dp))

In [12]:
inputs.cpu().numpy().shape == inputs2.cpu().numpy().shape

True

In [13]:
targets.cpu().numpy().shape == labels.cpu().numpy().shape

True

In [14]:
from cityscapes_helper import CitySpaceseDataset

In [15]:
data_test = CitySpaceseDataset('B:/Notes/SEM 3/EE641/Project/Cityscapes/')

In [16]:
test_loader =  DataLoader(data_train, batch_size=5, shuffle=True, drop_last=True, num_workers=0, pin_memory=False, generator=torch.Generator(device='cuda'))

In [17]:
inputs, targets = next(iter(train_loader))

In [18]:
inputs.cpu().size()

torch.Size([5, 256, 455, 3])

In [19]:
targets.cpu().size()

torch.Size([5, 256, 455])

In [20]:
for inputs, targets in dp:
    print(inputs.cpu().size())
    print(targets.cpu().size())

torch.Size([5, 3, 256, 455])
torch.Size([5, 256, 455])


In [130]:
def CEL(pred,target):
    criterion = nn.CrossEntropyLoss()
    pred = torch.squeeze(pred)
    target = torch.squeeze(target)
    loss = criterion(pred,target)
    return loss
def MSE(pred,target):
    criterion = nn.MSELoss()
    loss = criterion(pred,target)
    return loss 
def NLL(pred,target):
    criterion = nn.NLLLoss
    loss = criterion(pred,target)
def train(model, train_loader, optimizer, scheduler,  cfg=None, test_loader = None):  #writer
    best_loss = 1e10
    best_val = 100
    count = 0

    # looping over given number of epochs
    for epoch in range(1):
        tic = time.time()
        # scheduler.step()
        model.train()

        for inputs, targets in train_loader:
            count += 1

            inputs = inputs.cpu().type(torch.float32)
            inputs = np.transpose(inputs, [0,3,1,2]) 
            targets = targets.cpu().type(torch.float32)
            targets = targets[:,None,:,:]
#             print(targets.dtype, targets.size())
            
            
#             optimizer.zero_grad()
#             preds = model(inputs)
#             print(preds.dtype,preds.size())
#             loss = CEL(preds, targets)

#             loss.backward()
#             optimizer.step()

            # tensorboard logging
#             writer.add_scalar('Train/Loss', loss.item(), count)
            
            # Augmented data training
            aug_data = augBlock(inputs.cpu().numpy(), targets.cpu().numpy())
            aug_inputs, aug_targets = next(iter(aug_data))
            
            aug_inputs = aug_inputs.cpu()
            aug_targets = aug_targets.cpu()

            optimizer.zero_grad()
            aug_preds = model(aug_inputs)
            loss = CEL(aug_preds, aug_targets)

            loss.backward()
            optimizer.step()

            tensorboard logging
            writer.add_scalar('Train/Loss', loss.item(), count)

            print(f'Epoch:{epoch}, Step:{count}, Loss:{loss.item():.6f}, BestVal:{best_val:.6f}, Time:{time.time()-tic}')
            tic = time.time()
            
            if count == 2:
                break

    return loss.item()

In [131]:
from network import UNet as Model
model = Model()
model.cpu()
optimizer = optim.Adam(model.parameters(), lr=1e-4 )
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
val = train(model, train_loader, optimizer, scheduler, cfg=None, test_loader = None)

Epoch:0, Step:1, Loss:16327.634766, BestVal:100.000000, Time:21.555428743362427
Epoch:0, Step:2, Loss:15658.108398, BestVal:100.000000, Time:21.75443744659424


In [134]:
from torchmetrics import JaccardIndex, Dice

In [139]:
def dice(pred, targets):
    criterion = Dice(average='micro')
    loss = 0
    for i in range(len(pred)):
        print(targets[i][0].size())
        loss += criterion(torch.squeeze(pred[i][0].type(torch.int)),torch.squeeze(targets[i][0].type(torch.int)))
    return loss/len(pred)

def eval_model(model, test_loader,best_val=100, cfg=None):

    # Set model to evaluate mode
    model.eval()

    n_samples = 0
    avg_loss = 0
    count = 0
    # check dataset type
    for inputs, targets in test_loader:
        count += 1
        inputs = inputs.cpu().type(torch.float32)
        inputs = np.transpose(inputs, [0,3,1,2]) 
        targets = targets.cpu().type(torch.float32)
        targets = targets[:,None,:,:]

        with torch.set_grad_enabled(False):
            preds = model(inputs)  
#             preds = torch.clip(preds, 0, 1)
            print(preds.dtype,preds.size())
            loss = dice(preds, targets) 
            # NMSE
            
            print(f'Loss:{loss.item():.6f}')
#             avg_loss += (loss.item() * inputs.shape[0])
#             n_samples += inputs.shape[0]
        print(count)
        if count == 2:
            break

#     model.train()
    
    return loss.item()

In [141]:
eval_model(model, test_loader,best_val=100, cfg=None)

torch.float32 torch.Size([5, 1, 256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
Loss:0.019619
1
torch.float32 torch.Size([5, 1, 256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
torch.Size([256, 455])
Loss:0.009904
2


(0, 100)